In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import core as co
import numpy as np
import tensorflow as tf
import sklearn.metrics as metrics
import itertools

In [2]:
train_x = np.load('./datasets/data_allcuts/train_features.npy')
train_y = np.load('./datasets/data_allcuts/train_labels.npy')

val_x = np.load('./datasets/data_allcuts/val_features.npy')
val_y = np.load('./datasets/data_allcuts/val_labels.npy')

test_x = np.load('./datasets/data_allcuts/test_features.npy')
test_y = np.load('./datasets/data_allcuts/test_labels.npy')

In [3]:
print train_x.shape
print train_y.shape
print val_x.shape
print val_y.shape
print test_x.shape
print test_y.shape

(774, 60, 41, 2)
(774, 4)
(65, 60, 41, 2)
(65, 4)
(65, 60, 41, 2)
(65, 4)


In [4]:
frames = 41
bands = 60

num_labels = 4
num_channels = 2

batch_size = 50
kernel_size = 30
depth = 32
num_hidden = 200

starter_learning_rate = 1e-5
training_iterations = 10000
step = 300
decay = 0.75

In [5]:
X = tf.placeholder(tf.float32, shape=[None, bands, frames, num_channels])
Y = tf.placeholder(tf.float32, shape=[None, num_labels])
phase_train = tf.placeholder(tf.bool, name='phase_train')
global_step = tf.Variable(0, trainable=False)

conv1 = co.conv_layer(X, num_channels, depth, phase_train, 'conv1', kernel_size, kernel_size)
conv2 = co.conv_layer(conv1, depth, depth*2, phase_train, 'conv2', kernel_size, kernel_size)
max_pool_1 = co.max_pool_layer(conv2, name='max_pool_1')
shape = max_pool_1.get_shape().as_list()
max_pool_1_flat = tf.reshape(max_pool_1, [-1, shape[1] * shape[2] * shape[3]])

fc1 = co.fc_layer(max_pool_1_flat, shape[1] * shape[2] * shape[3], num_hidden, phase_train, 'fc1') # fc_layer fc1
y_ = co.fc_layer(tf.nn.relu(fc1), num_hidden, num_labels, phase_train, 'fc2')

1800
28800
40320
200


In [ ]:
learning_rate = tf.train.exponential_decay(starter_learning_rate,global_step,step,decay,staircase=True)

with tf.name_scope('xent'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_))
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy, global_step=global_step)
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
cost_history = np.empty(shape=[1],dtype=float)
val_cost_history = np.empty(shape=[1],dtype=float)
with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    for itr in range(training_iterations):    
        offset = (itr * batch_size) % (train_y.shape[0] - batch_size)
        batch_x = train_x[offset:(offset + batch_size), :, :, :]
        batch_y = train_y[offset:(offset + batch_size), :]
        
        _, c, a = session.run([optimizer, cross_entropy, accuracy],feed_dict={X: batch_x, Y : batch_y, phase_train : True})
        cost_history = np.append(cost_history,c)
        if itr % 1 == 0:
            vc, va = session.run([cross_entropy, accuracy],feed_dict={X: val_x, Y : val_y, phase_train : False})
            print ('Train cost', c, 'Accuracy', a, 'Validation cost', vc, 'Accuracy', va)
            val_cost_history = np.append(val_cost_history,vc)

    tc, ta = session.run([cross_entropy, accuracy], feed_dict={X: test_x, Y: test_y, phase_train: False})
    y_pred = session.run(y_, feed_dict={X: test_x, Y: test_y, phase_train: False})
    print('Test Cost: {} Test Accuracy: {}'.format(tc, ta))
    

('Train cost', 1.6970016, 'Accuracy', 0.02, 'Validation cost', 15.803811, 'Accuracy', 0.092307694)
('Train cost', 0.26952374, 'Accuracy', 1.0, 'Validation cost', 13.383987, 'Accuracy', 0.092307694)
('Train cost', 2.1379685, 'Accuracy', 0.68000001, 'Validation cost', 10.936047, 'Accuracy', 0.092307694)
('Train cost', 4.4616871, 'Accuracy', 0.0, 'Validation cost', 7.1176028, 'Accuracy', 0.092307694)
('Train cost', 3.1413665, 'Accuracy', 0.0, 'Validation cost', 4.5311399, 'Accuracy', 0.092307694)
('Train cost', 4.1686206, 'Accuracy', 0.0, 'Validation cost', 3.1157389, 'Accuracy', 0.092307694)
('Train cost', 5.0427995, 'Accuracy', 0.0, 'Validation cost', 2.6035433, 'Accuracy', 0.46153849)
('Train cost', 4.1195593, 'Accuracy', 0.0, 'Validation cost', 2.3762124, 'Accuracy', 0.46153849)
('Train cost', 3.3857582, 'Accuracy', 0.0, 'Validation cost', 2.2877882, 'Accuracy', 0.46153849)
('Train cost', 2.9575858, 'Accuracy', 0.0, 'Validation cost', 2.1684518, 'Accuracy', 0.46153849)
('Train cost', 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(15,10))
plt.plot(cost_history)
plt.plot(val_cost_history)
# plt.axis([0,training_epochs,0,np.max(cost_history)])
plt.show()

In [ ]:
y_test_class = np.argmax(test_y, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
conf_mat = metrics.confusion_matrix(y_test_class, y_pred_class)

In [ ]:
# Compute ROC curve and ROC area for each class
n_classes = 4
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(test_y[:, i], y_pred[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = metrics.roc_curve(test_y.ravel(), y_pred.ravel())
roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

plt.figure()
lw = 2
plt.plot(fpr["micro"], tpr["micro"], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc["micro"])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

roc_score = metrics.roc_auc_score(test_y, y_pred)

#print "Loss: {} Accuracy: {}%".format(loss_and_metrics[0],loss_and_metrics[1] * 100)
print "ROC AUC Score: ", roc_score

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
plot_confusion_matrix(conf_mat, classes=['A', 'B', 'C', 'D'])